### **환경 설정**

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.2 MB/s eta 0:16:26tcmalloc: large alloc 1147494400 bytes == 0x39dfc000 @  0x7f28a900d615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████▌               | 1055.7 MB 1.2 MB/s eta 0:13:10tcmalloc: large alloc 1434370048 bytes == 0x7e452000 @  0x7f28a900d615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████           | 1336.2 MB 1.3 MB/s eta 0:09:19tcmalloc: large alloc 1792966656 bytes =

In [ ]:
from mmcv import collect_env
collect_env()

{'CUDA available': True,
 'CUDA_HOME': '/usr/local/cuda',
 'GCC': 'x86_64-linux-gnu-gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0',
 'GPU 0': 'Tesla P100-PCIE-16GB',
 'MMCV': '1.5.0',
 'MMCV CUDA Compiler': '11.1',
 'MMCV Compiler': 'GCC 7.3',
 'NVCC': 'Cuda compilation tools, release 11.1, V11.1.105',
 'OpenCV': '4.1.2',
 'PyTorch': '1.9.0+cu111',
 'PyTorch compiling details': 'PyTorch built with:\n  - GCC 7.3\n  - C++ Version: 201402\n  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v2.1.2 (Git Hash 98be7e8afa711dc9b66c8ff3504129cb82013cdb)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - NNPACK is enabled\n  - CPU capability usage: AVX2\n  - CUDA Runtime 11.1\n  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-genco

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.9.0+cu111 True
2.24.1
11.1
GCC 7.3


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


### **데이터셋 다운로드**

In [ ]:
# Download dataset
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="MY_API_KEY")
project = rf.workspace("sketch").project("sketch-detection")
dataset = project.version(2).download("coco")

     |████████████████████████████████| 145 kB 5.1 MB/s 
     |████████████████████████████████| 178 kB 41.6 MB/s 
     |████████████████████████████████| 1.1 MB 66.5 MB/s 
     |████████████████████████████████| 67 kB 6.6 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 138 kB 63.9 MB/s 
     |████████████████████████████████| 596 kB 60.2 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for roboflow: filename=roboflow-0.2.4-py3-none-any.whl size=21356 sha256=c0aaf5e99e33b0fb3803edc77093b09ff15327d751f92774cc9ccf3efc530c80
  Stored in directory: /root/.cache/pip/wheels/b7/2c/60/3733c0e323b1d7aec00f852264314bf6c48192c8e1dc92b3ee
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=36dd495cf52f6e72b4a65bbb7449442e1d78bff738cf3184686d0ea677d387f0
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built roboflow wget

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to sketch-detection-2 in coco:: 100%|██████████| 3309/3309 [00:02<00:00, 1116.27it/s]


### **YoloF**

<img src="https://user-images.githubusercontent.com/40661020/144001639-257374ef-7d4f-412b-a783-88abdd22f277.png" width=500/>

In [ ]:
from mmcv import Config
from mmdet.apis import set_random_seed

cfg = Config.fromfile('./configs/yolof/yolof_r50_c5_8x8_1x_coco.py')

print(cfg.pretty_text)

dataset_type = 'CocoDataset'
data_root = 'data/coco/'
img_norm_cfg = dict(
    mean=[103.53, 116.28, 123.675], std=[1.0, 1.0, 1.0], to_rgb=False)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='RandomShift', shift_ratio=0.5, max_shift_px=32),
    dict(
        type='Normalize',
        mean=[103.53, 116.28, 123.675],
        std=[1.0, 1.0, 1.0],
        to_rgb=False),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(
                type='Normal

In [ ]:
# config 설정

data_root = '/content/mmdetection/sketch-detection-2/'
ann_file = '_annotations.coco.json'


cfg.data_root = data_root

cfg.data.train.ann_file = data_root + 'train/' + ann_file
cfg.data.train.img_prefix = data_root + 'train/'
cfg.data.train.classes = (
                      'button', 'checkbox_checked', 'checkbox_unchecked',
                      'chip', 'data_table', 'dropdown_menu',
                      'floating_action_button', 'image', 'radiobutton_checked',
                      'radiobutton_unchecked', 'slider', 'switch_disabled',
                      'switch_enabled', 'text_field', 'text_area')

cfg.data.val.ann_file = data_root + 'valid/' + ann_file
cfg.data.val.img_prefix = data_root + 'valid/'
cfg.data.val.classes = (
                      'button', 'checkbox_checked', 'checkbox_unchecked',
                      'chip', 'data_table', 'dropdown_menu',
                      'floating_action_button', 'image', 'radiobutton_checked',
                      'radiobutton_unchecked', 'slider', 'switch_disabled',
                      'switch_enabled', 'text_field', 'text_area')

cfg.data.test.ann_file = data_root + 'test/' + ann_file
cfg.data.test.img_prefix = data_root + 'test/'
cfg.data.test.classes = (
                      'button', 'checkbox_checked', 'checkbox_unchecked',
                      'chip', 'data_table', 'dropdown_menu',
                      'floating_action_button', 'image', 'radiobutton_checked',
                      'radiobutton_unchecked', 'slider', 'switch_disabled',
                      'switch_enabled', 'text_field', 'text_area')

cfg.data.samples_per_gpu = 2
cfg.data.workers_per_gpu = 2

cfg.optimizer.lr = 0.0025

cfg.model.bbox_head.num_classes = 15

cfg.runner.max_epochs = 30
cfg.max_epochs = 30

cfg.work_dir = 'working/yolof'

cfg.seed = 0

cfg.gpu_ids = [0]
cfg.device = 'cuda'

cfg.log_config.interval = 100

In [24]:
print(cfg.pretty_text)

# config 파일 저장
with open("yolof.py","w") as f:
  f.write(cfg.pretty_text)

!cp yolof.py /content/drive/MyDrive/model/yolof

dataset_type = 'CocoDataset'
data_root = '/content/mmdetection/sketch-detection-2/'
img_norm_cfg = dict(
    mean=[103.53, 116.28, 123.675], std=[1.0, 1.0, 1.0], to_rgb=False)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='RandomShift', shift_ratio=0.5, max_shift_px=32),
    dict(
        type='Normalize',
        mean=[103.53, 116.28, 123.675],
        std=[1.0, 1.0, 1.0],
        to_rgb=False),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict

In [21]:
# 학습

import os.path as osp

from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

# Save checkpoints
!cp -r /content/mmdetection/working/yolof /content/drive/MyDrive/model

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


2022-05-14 09:38:54,736 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-05-14 09:38:54,766 - mmdet - INFO - Start running, host: root@7733bb150349, work_dir: /content/mmdetection/working/yolof
2022-05-14 09:38:54,767 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook     

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


2022-05-14 09:39:09,576 - mmdet - INFO - Epoch [1][100/1352]	lr: 1.666e-04, eta: 1:39:25, time: 0.147, data_time: 0.029, memory: 6805, loss_cls: 4.9721, loss_bbox: 0.3820, loss: 5.3541
2022-05-14 09:39:22,085 - mmdet - INFO - Epoch [1][200/1352]	lr: 3.331e-04, eta: 1:31:39, time: 0.125, data_time: 0.008, memory: 6805, loss_cls: 0.6724, loss_bbox: 0.3647, loss: 1.0371
2022-05-14 09:39:34,594 - mmdet - INFO - Epoch [1][300/1352]	lr: 4.997e-04, eta: 1:28:55, time: 0.125, data_time: 0.008, memory: 6805, loss_cls: 0.5828, loss_bbox: 0.3717, loss: 0.9544
2022-05-14 09:39:47,849 - mmdet - INFO - Epoch [1][400/1352]	lr: 6.662e-04, eta: 1:28:42, time: 0.133, data_time: 0.009, memory: 6805, loss_cls: 0.5536, loss_bbox: 0.3840, loss: 0.9377
2022-05-14 09:40:00,353 - mmdet - INFO - Epoch [1][500/1352]	lr: 8.328e-04, eta: 1:27:29, time: 0.125, data_time: 0.008, memory: 6805, loss_cls: 0.5336, loss_bbox: 0.4109, loss: 0.9445
2022-05-14 09:40:12,864 - mmdet - INFO - Epoch [1][600/1352]	lr: 9.993e-04,

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 298/298, 24.0 task/s, elapsed: 12s, ETA:     0s

...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 298/298, 22.7 task/s, elapsed: 13s, ETA:     0s

2022-05-14 11:12:40,785 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


2022-05-14 11:12:41,418 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.811
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.889
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.889
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.811
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.919
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.919
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.919
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=

DONE (t=0.46s).
Accumulating evaluation results...
DONE (t=0.15s).
